（1）ortools结果转为自己的表示，类似于output里写的东西

（2）评估解（用一个类描述VRP问题）

（3）定义扰动

（4）模拟退火（或其他元启发式）

In [6]:
import random

In [ ]:
solution_code=[[]]#[[15,3,8,0,6,5,11,4,0,...],...,[0,0,5,3,0,1,0,0]，用0分割不同车，按下标排序;最后一趟可能有很多0

In [ ]:
def exchangesolution(nodes, goods, ports, vehicles, manager, routing, solution):
    '''输入ortools的solution,返回我们定义的solution_code'''
    changed_solution = []
    for vehicle_index in range(vehicles.number):
        index = routing.Start(vehicle_index)
        previous_index = index
        while not routing.IsEnd(index):
            if index != 0:
                changed_solution.append(index)
            index = solution.Value(routing.NextVar(index))
        if vehicle_index != vehicles.number-1:
            changed_solution.append(0)
    return changed_solution

In [ ]:
class VRP():
    def __init__(self,nodes,goods,vehicles,time_limit):
        self.nodes = nodes
        self.goods = goods
        self.vehicles = vehicles
        self.time_limit = time_limit    # [time]*vehicles.number, 每辆车的时间约束
        self.s = 60*10          # 停留时间
        
    def evaluate(self,solution):
        '''
        输入solution_code,返回目标函数值，
        现在目标是总距离，返回总距离'''
        total_dist = 0
        idx = [-1]+[i for i,x in enumerate(solution) if x==0]+[len(solution)]   # 0的位置
        for i in range(1, len(idx)):
            route = [0]+solution[idx[i-1]+1:idx[i]]+[0]
            if len(route) == 2:
                continue
            for j in range(1, len(route)):
                total_dist += self.nodes.dist_matrix[route[j-1]][route[j]]
        return total_dist
    
    def check_constraint(self,solution):
        '''输入solution_code,返回bool变量，判断解是否满足约束'''
        flag = True
        idx = [-1]+[i for i,x in enumerate(solution) if x==0]+[len(solution)]   # 0的位置
        for i in range(1, len(idx)):
            route = [0]+solution[idx[i-1]+1:idx[i]]+[0]
            if len(route) == 2:
                continue
            total_v = 0
            total_time = 0
            for j in range(1, len(route)):
                total_time += self.nodes.time_matrix[route[j-1]][route[j]]
                if route[j] != 0:
                    total_time += self.s
                    total_v += self.nodes.nodes.v_total.iloc[route[j]]
            # 检查时间约束
            if total_time > self.time_limit[i]:
                flag = False
                break
            # 检查车辆容积约束
            if total_v > self.vehicles.vehicles.capacity.iloc[i]:
                flag = False
                break
        return flag
    
    def get_result(self,solution):
        '''输入solution_code,和其他需要的信息,返回满足json格式的dict变量，最后有空再写'''
        arrive_time = [0] * self.nodes.goods_number  # 每个货物到达其目标驿站的时间
        vehicle_trajectory = [0] * self.vehicles.number  # 车辆运行轨迹
        vehicle_timetable = [0] * self.vehicles.number  # 车辆时间表
        load_sequence = [0] * self.vehicles.number  # 货物装载顺序
        vehicle_dist = [0] * self.vehicles.number  # 车辆移动距离
        for i in range(1, len(idx)):
            route = [0]+solution[idx[i-1]+1:idx[i]]+[0]
            if len(route) == 2:
                vehicle_trajectory[i]= [0]
                vehicle_timetable[i]= [0]
                load_sequence[i] = [[]]
                vehicle_dist[i] = [0]
                continue
            route_time = 0
            route_dist = 0
            vehicle_trajectory[i]= [0]
            vehicle_timetable[i]= [0]
            vehicle_dist[i] = [0]
            for j in range(1, len(route)):
                from_node = self.nodes.nodes.iloc[route[j-1]]
                to_node = self.nodes.nodes.iloc[route[j]]
                route_time += self.nodes.time_matrix[route[j-1]][route[j]]
                route_dist += self.nodes.dist_matrix[route[j-1]][route[j]]
                if from_node.n_port == to_node.n_port:
                    seq = load_sequence[i][-1]
                    for good_idx in to_node.goodset:
                        seq.append(self.goods.IndexToId[good_idx])
                    load_sequence[i][-1] = seq
                else:
                    vehicle_trajectory[i].append(to_node.n_port)
                    vehicle_timetable[i].append(route_time)
                    vehicle_dist[i].append(route_dist)
                    seq = []
                    for good_idx in to_node.goodset:
                        seq.append(self.goods.IndexToId[good_idx])
                    load_sequence[i].append(seq)
                route_time += self.s
            
        result = dict()
        result["orderList"] = []
        for i in range(len(arrive_time)):
            if arrive_time[i] != 0:
                order = dict()
                order["id"] = self.goods.IndexToId[i]
                order["arrivalTime"] = arrive_time[i]
                result["orderList"].append(order)
        result["carList"] = []
        for vehicle_index in range(len(vehicle_trajectory)):
            if len(vehicle_trajectory[vehicle_index]) <= 1:
                continue
            car = dict()
            car["id"] = self.vehicles.IndexToId[vehicle_index]
            car["returnTime"] = vehicle_timetable[vehicle_index][-1]
            car["orderOrder"] = []
            for i in range(len(load_sequence[vehicle_index])):
                car["orderOrder"] = car["orderOrder"]+load_sequence[vehicle_index][i]
            car["route"]= []
            for index, port_id in enumerate(vehicle_trajectory[vehicle_index]):
                port = dict()
                port["siteId"] = int(port_id[1])
                port["type"] = int(port_id[0])
                port["arrivalTime"] = vehicle_timetable[vehicle_index][index]
                port["orderIds"] = load_sequence[vehicle_index][index]
                # port["dist"] = vehicle_dist[vehicle_index][index]
                car["route"].append(port)
            result["carList"].append(car)
        return result
        

In [ ]:
def ls_swap(solution):
    #交换
    idx1 = random.randint(0, len(solution)-1)
    while(solution[idx1] != 0):
        idx1 = random.randint(0, len(solution)-1)
    idx2 = random.randint(0, len(solution)-1)
    while(solution[idx2] != 0):
        idx2 = random.randint(0, len(solution)-1)
    new_solution = solution.copy()
    new_solution[idx1] = solution[idx2]
    new_solution[idx2] = solution[idx1]
    return new_solution

In [12]:
import numpy as np
random.randint(0,5)

1

In [ ]:
def sa(initials,tmin=0.05,t=35,alpha=0.94,limit=200):
    '''模拟退火，以前我写过，我找一下。
    initials:初始solution_code
    limit:多少次没找到更优解就终止运行了'''
    ini_fun = vrp.evaluate(initials)
    best_fun = ini_fun
    fun=ini_fun
    best_code = initials
    solution = initials
    count=0
    while t > tmin:
        while True:
            new_solution=ls_swap(solution)
            if vrp.check_constraint(new_solution):
                break
        new_fun=vrp.evaluate(new_solution)
        de=fun-new_fun
        if de > 0 & (np.random.rand() > np.exp(-de / t)):
            solution=new_solution
            fun=new_fun
            if fun < best_fun:
                count=0
                best_fun=fun
                best_code=solution
            else:
                count+=1
        if count>=limit:
            break
        t = t * alpha
    return best_code,best_fun